### 1 - Import test data

In [35]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-02-24", end="2024-04-14", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-02-26 00:00:00,1.082251,1.082251,1.082134,1.082134,1.082134,0
2024-02-26 00:15:00,1.082251,1.082251,1.081900,1.082017,1.082017,0
2024-02-26 00:30:00,1.082134,1.082251,1.082134,1.082134,1.082134,0
2024-02-26 00:45:00,1.082134,1.082251,1.082017,1.082017,1.082017,0
2024-02-26 01:00:00,1.081900,1.081900,1.081666,1.081783,1.081783,0
...,...,...,...,...,...,...
2024-04-12 21:15:00,1.064623,1.064623,1.064509,1.064509,1.064509,0
2024-04-12 21:30:00,1.064509,1.064623,1.064396,1.064509,1.064509,0
2024-04-12 21:45:00,1.064509,1.064509,1.064396,1.064396,1.064396,0


### 2 - Define your signal function

In [36]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal
dataF

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2024-02-26 00:00:00,1.082251,1.082251,1.082134,1.082134,1.082134,0,0
2024-02-26 00:15:00,1.082251,1.082251,1.081900,1.082017,1.082017,0,0
2024-02-26 00:30:00,1.082134,1.082251,1.082134,1.082134,1.082134,0,0
2024-02-26 00:45:00,1.082134,1.082251,1.082017,1.082017,1.082017,0,0
2024-02-26 01:00:00,1.081900,1.081900,1.081666,1.081783,1.081783,0,0
...,...,...,...,...,...,...,...
2024-04-12 21:15:00,1.064623,1.064623,1.064509,1.064509,1.064509,0,0
2024-04-12 21:30:00,1.064509,1.064623,1.064396,1.064509,1.064509,0,0
2024-04-12 21:45:00,1.064509,1.064509,1.064396,1.064396,1.064396,0,0


In [37]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    3091
2     111
1     108
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [38]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import oandapyV20.endpoints.instruments as instruments

In [31]:
#from config import access_token, accountID
access_token='f75cde3e8244c96dd2f9515b575bb8a1-0149b3a4f5f84c4efd98459ae49110bf'
def get_candles(n):

    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [32]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    accountID = "101-001-17799329-001" 
    client = API(access_token)
         
    SLTPRatio = 2
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [34]:
trading_job()

scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='sun-fri', hour='00-23', minute='1,16,31,46', start_date='2024-01-12 12:00:00', timezone='America/Chicago')
scheduler.start()

      Open    Close     High      Low
0  1.06350  1.06335  1.06353  1.06322
1  1.06339  1.06412  1.06423  1.06337
1.0658200000000002    1.06324    1.0623799999999999    1.0649600000000001


ValueError: The minimum value in a range must not be higher than the maximum